In [1]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

In [2]:
df = pd.read_csv('template_and_user_query.csv')

In [4]:
val_df1 = df.iloc[::9,:]
val_df2 = df.iloc[::10,:]

In [5]:
val_df = pd.concat([val_df1, val_df2])

In [6]:
val_df.reset_index(drop=True, inplace=True)

In [9]:
df = pd.concat([df,val_df]).drop_duplicates(keep=False).reset_index()

In [10]:
df.drop(columns=['index'], inplace=True)

In [11]:
#val_df.to_csv('val_df.csv', index=False)

In [12]:
#df = df.sample(frac=1).reset_index(drop=True)

In [13]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/llama-2-7b-chat-hf"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
new_model = "llama-2-7b-ft-peft-on-template_and_user_query-data"

In [15]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [16]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [17]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/data/sept-23"

# Number of training epochs
num_train_epochs = 15

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 1000

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 30

# Log every X updates steps
logging_steps = 30

In [18]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [19]:
prompt_template1 = """### _LUMIN_ Question:
{question}
### Context: {context}
### _LUMIN_ Answer: {answer}"""

In [20]:
promt_template_v2 = """[INST]<<SYS>>
You are an advanced template converter that converts user question to a specific template which answers the user question.

<</SYS>>

{user_query}
[/INST]
[LUMINTEMPLATE]
{template_query}
[/LUMINTEMPLATE]</s>"""

In [21]:
prompt_template = """### Question:
{user_query}

### Context:
Converting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer.

### Answer:
{template_query}"""

In [22]:
context = "Converting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer."

In [23]:
df.columns

Index(['question', 'user query'], dtype='object')

In [24]:
def create_fine_tuning_dataset(row):
    user_query = row['user query']
    template_query = row['question']
    formated = promt_template_v2.format(user_query=user_query,
                                      template_query=template_query)
    # data_point = str({'question': formated, "answer":template_query})
    return formated

In [25]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [26]:
val_df['fine_tuning_dataset']=val_df.apply(create_fine_tuning_dataset, axis=1)

In [27]:
val_df['fine_tuning_dataset'][0]

'[INST]<<SYS>>\nYou are an advanced template converter that converts user question to a specific template which answers the user question.\n\n<</SYS>>\n\nWhat were the 2020 sales figures?\n[/INST]\n[LUMINTEMPLATE]\nWhat is the sales in 2020\n[/LUMINTEMPLATE]</s>'

In [28]:
df['fine_tuning_dataset'][0]

'[INST]<<SYS>>\nYou are an advanced template converter that converts user question to a specific template which answers the user question.\n\n<</SYS>>\n\nHow much were the sales in 2020?\n[/INST]\n[LUMINTEMPLATE]\nWhat is the sales in 2020\n[/LUMINTEMPLATE]</s>'

In [29]:
df.drop(columns=['question','user query'], inplace=True)

In [30]:
df.columns

Index(['fine_tuning_dataset'], dtype='object')

In [31]:
val_df.drop(columns=['question','user query'], inplace=True)

In [32]:
val_df.shape

(503, 1)

In [33]:
train_dataset = Dataset.from_pandas(df)
val_dataset = Dataset.from_pandas(val_df)

In [34]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 1898
})

In [35]:
val_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 503
})

In [36]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [37]:
bnb_4bit_quant_type

'nf4'

In [38]:
compute_dtype

torch.float16

In [39]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [40]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [42]:
max([len(tokenizer.encode(df['fine_tuning_dataset'][i])) for i in range(1800)])

123

In [43]:
tokenizer.decode([2])

'</s>'

In [44]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [45]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    eval_steps=30,
    per_device_eval_batch_size=1, # Batch size for evaluation
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=1000,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

In [46]:
## Getting FLOPs of model

model_flops = (
  model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 512)
      )
    }
  )
  * training_arguments.gradient_accumulation_steps
)

#print(model)
print("Memory footprint", model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

Memory footprint 3.829936128 GB
Flops 10350.615330816 GFLOPs


In [47]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 1898
})

In [48]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.8/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1898 [00:00<?, ? examples/s]

Map:   0%|          | 0/503 [00:00<?, ? examples/s]

In [49]:
train_dataset[0]

{'fine_tuning_dataset': '[INST]<<SYS>>\nYou are an advanced template converter that converts user question to a specific template which answers the user question.\n\n<</SYS>>\n\nHow much were the sales in 2020?\n[/INST]\n[LUMINTEMPLATE]\nWhat is the sales in 2020\n[/LUMINTEMPLATE]</s>'}

In [ ]:
# Train model
trainer.train()

Step,Training Loss,Validation Loss
30,0.890100,0.737551
60,0.434600,0.368406


In [ ]:
# Fine-tuned model name
new_model = "llama-2-7b-ft-peft-v4-lr-64-new-template"

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [41]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_

In [43]:
from peft import PeftModel, PeftConfig

In [49]:
model_to_merge = PeftModel.from_pretrained(
    model,
    'llama-2-7b-fine-tuned-peft-v2')

In [126]:
model_to_merge = PeftModel.from_pretrained(
    model,
    '/data/sept-23/refract-1695579045682/checkpoint-900')

In [127]:
## Getting FLOPs of model

model_flops = (
  model_to_merge.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 200)
      )
    }
  )
  * training_arguments.gradient_accumulation_steps
)

#print(model)
print("Memory footprint", model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

Memory footprint 4.488974336 GB
Flops 4083.474432 GFLOPs


In [45]:
query_template = """### Question:
{user_query}

### Context:
Converting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer.

"""

In [128]:
query_template_v2 = """[INST]<<SYS>>
You are an advanced template converter that converts user question to a specific template which answers the user question.

<</SYS>>

{user_query}
[/INST]
[LUMINTEMPLATE]"""

In [129]:
model_to_merge.to('cuda')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (

In [130]:
def predict_template_query_v3(user_query):
    inp = query_template_v2.format(user_query=user_query)
    _inputs = tokenizer.encode(inp, return_tensors="pt")
    outputs = model_to_merge.generate(input_ids=_inputs.to('cuda'), max_length= 180)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[LUMINTEMPLATE]\n')[1]
    return output_new.split('\n[/LUMINTEMPLATE]')[0]

In [132]:
%%time
predict_template_query_v3("Growth rate of sales share of phone")

CPU times: user 1min 4s, sys: 30.3 s, total: 1min 34s
Wall time: 1min 34s


'What is the growth contribution to overall sales for phone'

In [133]:
%%time
predict_template_query_v3("What is the growth contribution to overall sales for phone in 2021")

CPU times: user 1min 2s, sys: 28.8 s, total: 1min 31s
Wall time: 1min 31s


'What is the growth contribution to overall sales for phone in 2021'

In [134]:
%%time
predict_template_query_v3("Which are the top 3 sub category based on share/contribution of sales across segment except paper")

CPU times: user 59.8 s, sys: 27.6 s, total: 1min 27s
Wall time: 1min 27s


'Which are the top 3 sub category based on contribution to overall sales except paper'

In [135]:
%%time
predict_template_query_v3("which are the 5 top selling sub category by sales in 2023")

CPU times: user 59.3 s, sys: 26.9 s, total: 1min 26s
Wall time: 1min 26s


'Which are the top sub category basis sales in 2023'

In [136]:
%%time
predict_template_query_v3("how does the sales change for phone in the last year")

CPU times: user 1min 4s, sys: 29.4 s, total: 1min 33s
Wall time: 1min 33s


'What is the trend of sales for phone in 2020'

In [137]:
%%time
predict_template_query_v3("what is the average monthly market share of paper for the last two and a half years")

CPU times: user 1min 2s, sys: 28.6 s, total: 1min 30s
Wall time: 1min 30s


'What is the average monthly share of paper sales over the last 2.5 years'

In [138]:
%%time
predict_template_query_v3("when was the monthly selling of paper recorded the lowest?")

CPU times: user 1min 3s, sys: 29 s, total: 1min 32s
Wall time: 1min 32s


'When was the last time that sales of paper across months was lowest'

In [49]:
%%time
predict_template_query_v3("What were the sales in '23?")

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2415: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


CPU times: user 23.8 s, sys: 5.03 s, total: 28.8 s
Wall time: 28.8 s


'What is the sales in 2023'

In [50]:
%%time
predict_template_query_v3("sales number in year 2022?")

CPU times: user 22.7 s, sys: 5.03 s, total: 27.8 s
Wall time: 27.8 s


'What is the sales in 2022'

In [53]:
%%time
predict_template_query_v3("what was profit for the year 2020?")

CPU times: user 1min 4s, sys: 30.3 s, total: 1min 34s
Wall time: 1min 34s


'What is the sales in 2020'

In [49]:
%%time
predict_template_query_v3("what was sales in 11/20?")

CPU times: user 1min 10s, sys: 33.4 s, total: 1min 43s
Wall time: 1min 43s


'What is the sales in 2020'

In [51]:
%%time
predict_template_query_v3("Compare 1983 and 1984 sales and discount percentages.")

CPU times: user 1min 2s, sys: 29.1 s, total: 1min 31s
Wall time: 1min 31s


'What is the sales and discount percentage in 1983 vs 1984'

In [54]:
%%time
predict_template_query_v3("what was sales in twenty twenty two?")

CPU times: user 23.9 s, sys: 5.52 s, total: 29.4 s
Wall time: 29.4 s


'What is the sales in 2022'

In [52]:
%%time
predict_template_query_v3("what are profit each months")

CPU times: user 1min 7s, sys: 31.3 s, total: 1min 38s
Wall time: 1min 38s


'What is the monthly trend of profit'

In [45]:
%%time
predict_template_query_v3("When did the previous low in phone sales take place?")

CPU times: user 1min 5s, sys: 29.9 s, total: 1min 35s
Wall time: 1min 35s


'When was the last time that sales of phone was lowest'

In [46]:
%%time
predict_template_query_v3("When was phone sales lowest recently?")

CPU times: user 1min 8s, sys: 31.8 s, total: 1min 40s
Wall time: 1min 40s


'When was the last time that sales of phone was lowest'

In [67]:
%%time

inp = 'what are you doing'
inp = tokenizer.encode(inp, return_tensors='pt')
output = model.generate(input_ids=inp.to('cuda'), max_length= 200)
output = tokenizer.decode(outputs[0])

CPU times: user 1min 47s, sys: 50 s, total: 2min 37s
Wall time: 2min 37s


In [59]:
from tqdm import tqdm

In [56]:
df2 = pd.read_csv('template_and_user_query.csv')

In [57]:
df2_new = df2[300:500]

In [60]:
%%time
output_list_300_500 = []
for i, row in tqdm(df2_new.iterrows()):
    inp = row['user query']
    try:
        output = predict_template_query_v3(inp)
    except:
        output = None
        print(f"Got exception while processing : {inp}")
    output_list_300_500.append(output)

200it [1:42:11, 30.66s/it]

CPU times: user 1h 21min 28s, sys: 20min 42s, total: 1h 42min 10s
Wall time: 1h 42min 11s


In [62]:
df = pd.DataFrame(output_list_300_500, columns=['predicted_template'])
df.to_csv('predict_df_300_500.csv', index=False)

In [61]:
output_list_300_500

['What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper in 2020',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the sales of phone and paper across months',
 'What is the s

In [86]:
output_list_200_300

['What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category across months',
 'What is the sales and discount percentage by sub category and country across months',
 'What is the sales and discount percentage by sub category and country across months',
 'What is the sales and discount percentage by sub category and 

In [81]:
output_list_100_200

['What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage by sub category and country in 2020',
 'What is the sales and discount percentage in 2020 vs 2021',
 'What is the sales and discount percentage in 2020 vs 2021',
 'What is the sales and discount percentage in 2020 vs 2

In [78]:
output_list

['What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales across months in 2020',
 'What is the sales by sub category in 2020',
 'What is the sales by sub category in 2020',
 'What is the sales by sub category in 2020',
 'What is the sales by sub category in 2020',
 'What is the sales by sub category in 2020',
 'What is the sales by sub cat

In [40]:
def predict_template_query(user_query):
    question = query_template.format(user_query=user_query)
    _input = str({'question': question})
#     print(_input)
    _inputs = tokenizer.encode(_input, return_tensors="pt")
#     print(_inputs)
#     print(type(_inputs))
    outputs = model_to_merge.generate(input_ids=_inputs.to('cuda'), max_length= 200)
    output = tokenizer.decode(outputs[0])
    # print(output)
    # a = output.split("_LUMIN_ Answer",1)[1]
    #print(a)
    return output

In [ ]:
model_to_merge.to('cuda')

In [63]:
def process_output(a):
    b=a.split("### Answer",1)[1]
    return b.split("\n")[1]

In [5]:
# inp = "why production dropped for bikes in 1st quarter 2022"
# predict_template_query(inp)

In [4]:
# inp = 'why sales of texas increase in 1st quarter 2023'
# a = predict_template_query(inp)

In [2]:
# inp = 'drivers of profit'
# predict_template_query(inp)

In [3]:
# inp = "why sales dropped for bikes in 1st quarter 2022"
# predict_template_query(inp).split('\n')[1]